# Collect data for github action workflow runs

In this notebook, we collect historical test data like the test duration values from running workflows on Github using the GitHub API

## Collect data for selected workflow runs of a repository

From historical test workflow runs, want to extract
- time durations
- workflow run status & conclusion

We can get workflow IDs of the test that we are interested in from https://api.github.com/repos/{ORG}/{REPO}/actions/workflows 

In [1]:
from dotenv import find_dotenv, load_dotenv
import os
import json
import subprocess
from datetime import datetime
from subprocess import PIPE
import pandas as pd
from sklearn.model_selection import train_test_split

pd.options.mode.chained_assignment = None

import warnings

warnings.filterwarnings("ignore")

In [2]:
load_dotenv(find_dotenv(), override=True)
TOKEN = os.getenv("GITHUB_ACCESS_TOKEN")
ORG = os.getenv("ORG")
REPO = os.getenv("REPO")

In [3]:
# mode can be "check or "workflow" depending on whether you want to collect checks data or workflows data
MODE = "workflow"
test_id = "28698040" # Pre-commit test
# test_id = "29176617" # Build and push image
# test_id = "37149896" # File linting
# test_id = "37149895" # Spell Check
# test_id = "39411153" #"Compile all queries using the latest stable CodeQL CLI"

* Workflows: For example, lets collect data for the test ID 28698040 for the workflow runs in the repository `oss-aspen/8Knot` https://api.github.com/repos/oss-aspen/8Knot/actions/workflows

* Checks: Get commits for a repo and get check runs for each commit

In [4]:
def get_page_numbers(test_id):
    """
    Get the total count of tests.
    Find the pages on github-actions.
    """
    command = """curl \
      -H "Accept: application/vnd.github+json" \
      -H "Authorization: Bearer {}"\
      -H "X-GitHub-Api-Version: 2022-11-28" \
      https://api.github.com/repos/{}/{}/actions/workflows/{}/runs?""".format(TOKEN, ORG, REPO, test_id)
    args = []
    args.append(command)
    output = subprocess.run(args, shell=True, check=True, stdout=PIPE, stderr=PIPE)
    output = json.loads(output.stdout)
    total_count = output['total_count']
    page_numbers = int(total_count/30) # by default number of tests on one page is 30
    return page_numbers

In [5]:
def get_workflow_runs(test_id, page_numbers):
    """
    This function takes test_id and number of pages of workflow runs as input.
    Interacts with github api and collects the data for the tests with the specified id.
    Outputs the data frame with test data.
    """
    for p in range(1,page_numbers+1):
        command = """curl \
      -H "Accept: application/vnd.github+json" \
      -H "Authorization: Bearer {}"\
      -H "X-GitHub-Api-Version: 2022-11-28" \
      https://api.github.com/repos/{}/{}/actions/workflows/{}/runs?page={}""".format(TOKEN, ORG, REPO, test_id, p)
        args = []
        args.append(command)

        output = subprocess.run(args, shell=True, check=True, stdout=PIPE, stderr=PIPE)
        output = json.loads(output.stdout)

        if p==1:
            df = pd.json_normalize(output['workflow_runs'])
        else:
            df2 = pd.json_normalize(output['workflow_runs'])
            df = pd.concat([df, df2], axis=0)
    return df

In [6]:
def get_check_runs(commits):
    """
    This function takes test_id and number of pages of workflow runs as input.
    Interacts with github api and collects the data for the tests with the specified id.
    Outputs the data frame with test data.
    """
    appended_data = []
    for commit in commits:
        command = """curl -L \
          -H "Accept: application/vnd.github+json" \
          -H "Authorization: Bearer {}"\
          -H "X-GitHub-Api-Version: 2022-11-28" \
          https://api.github.com/repos/{}/{}/commits/{}/check-runs""".format(TOKEN, ORG, REPO, commit)
        args = []
        args.append(command)

        output = subprocess.run(args, shell=True, check=True, stdout=PIPE, stderr=PIPE)
        output = json.loads(output.stdout)

        appended_data.append(pd.json_normalize(output['check_runs']))

    df = pd.concat(appended_data, axis=0)

    return df

In [7]:
def get_commits():
    "This function gets all commit refs for a github repo"

    command =  """curl -L \
      -H "Accept: application/vnd.github+json" \
      -H "Authorization: Bearer {}"\
      -H "X-GitHub-Api-Version: 2022-11-28" \
      https://api.github.com/repos/{}/{}/commits""".format(TOKEN, ORG, REPO)
    args = []
    args.append(command)

    output = subprocess.run(args, shell=True, check=True, stdout=PIPE, stderr=PIPE)
    output = json.loads(output.stdout)

    df = pd.json_normalize(output)
    return df

In [8]:
if MODE == "check":
    # get checks
    df_commits = get_commits()
    commit_ids = list(df_commits['sha'])
    checks_df = get_check_runs(commit_ids)
    test_df = checks_df[['started_at', 'completed_at', 'id', 'status', 'conclusion', 'external_id', 'name']]
    test_df = test_df.rename(columns={'external_id': 'test_id', 'id': 'run_id'})
    test_df['run_duration'] = test_df.apply(lambda x: (datetime.strptime(x['completed_at'], "%Y-%m-%dT%H:%M:%SZ") - \
                                           datetime.strptime(x['started_at'], "%Y-%m-%dT%H:%M:%SZ")).total_seconds(), axis = 1)

elif MODE == "workflow":
    # get workflows
    page_numbers = get_page_numbers(test_id)
    workflow_df = get_workflow_runs(test_id, page_numbers)
    test_df = workflow_df[['created_at', 'updated_at', 'id', 'status', 'conclusion']]
    test_df = test_df.rename(columns={'id': 'run_id'})
    test_df['run_duration'] = test_df.apply(lambda x: (datetime.strptime(x['updated_at'], "%Y-%m-%dT%H:%M:%SZ") - \
                                           datetime.strptime(x['created_at'], "%Y-%m-%dT%H:%M:%SZ")).total_seconds(), axis = 1)
    test_df['test_id'] = test_id

In [9]:
test_df.head(10)

,created_at,updated_at,run_id,status,conclusion,run_duration,test_id
0,2023-03-21T16:47:52Z,2023-03-21T16:48:28Z,4481655529,completed,success,36.0,28698040
1,2023-03-02T18:04:52Z,2023-03-02T18:05:21Z,4316905413,completed,success,29.0,28698040
2,2023-02-28T22:46:09Z,2023-02-28T22:46:39Z,4298089675,completed,success,30.0,28698040
3,2023-02-28T22:42:47Z,2023-02-28T22:43:10Z,4298071211,completed,failure,23.0,28698040
4,2023-02-28T22:21:25Z,2023-02-28T22:21:46Z,4297918764,completed,success,21.0,28698040
5,2023-02-22T23:15:44Z,2023-02-22T23:16:07Z,4247826657,completed,success,23.0,28698040
6,2023-02-22T18:27:33Z,2023-02-22T18:28:00Z,4245797712,completed,success,27.0,28698040
7,2023-02-22T17:58:00Z,2023-02-22T17:58:29Z,4245558935,completed,success,29.0,28698040
8,2023-02-22T14:42:04Z,2023-02-22T14:42:26Z,4243845186,completed,success,22.0,28698040
9,2023-02-21T21:03:50Z,2023-02-21T21:04:14Z,4236840889,completed,success,24.0,28698040


In [10]:
test_df.shape

(270, 7)

In [11]:
# generating passing and failing dfs which are neccesary for computing fit distributions
passing_df = test_df[test_df['conclusion'] == 'success'] 
failures_df = test_df[test_df['conclusion'] == 'failure'] 

In [12]:
passing_df.shape

(122, 7)

In [13]:
failures_df.shape

(148, 7)

In [14]:
# Split the data into train and test.
passing_train, passing_test = train_test_split(passing_df, test_size=0.2, random_state=25)

In [15]:
# Split the data into train and test
failing_train, failing_test = train_test_split(failures_df, test_size=0.2, random_state=25)

In [16]:
passing_train.shape

(97, 7)

In [17]:
passing_test.shape

(25, 7)

In [18]:
failing_train.shape

(118, 7)

In [19]:
failing_test.shape

(30, 7)

In [20]:
passing_train.to_csv("../data/processed/{}passing_train.csv".format(test_id))
failing_train.to_csv("../data/processed/{}failing_train.csv".format(test_id))
passing_test.to_csv("../data/processed/{}passing_test.csv".format(test_id))
failing_test.to_csv("../data/processed/{}failing_test.csv".format(test_id))

## Conclusion

In this notebook, we interact with the github api to collect the data for all workflow runs. In future work, we will look into using this data to perform statistical tests using OSP model.